<a href="https://colab.research.google.com/github/Ambuka-nuru/Data/blob/main/MODIS_LST_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Background Information
# The farm plots were generated randomly using geojson.io and saved as a GeoJSON file. These plots were then merged using QGIS and uploaded to Google Earth Engine (GEE) as feature assets. The uploaded assets were used to define the study area for sampling MODIS Land Surface Temperature (LST) data.

# Temperature Data
# The temperature data in this project comes from the MODIS (Moderate Resolution Imaging Spectroradiometer) instrument on NASA's Terra satellite. The specific dataset used is the MODIS Land Surface Temperature and Emissivity Daily (MOD11A1) product, which provides daily LST at 1 km spatial resolution. The temperature values in the MOD11A1 dataset are provided in Kelvin and need to be converted to Celsius for easier interpretation.

# MODIS Dataset Used
# MODIS/061/MOD11A1: This dataset contains daily land surface temperature (LST) and emissivity data. The key parameters include:
# LST_Day_1km: Daytime land surface temperature at 1 km resolution.
# LST_Night_1km: Nighttime land surface temperature at 1 km resolution.
# The MOD11A1 dataset provides temperature values in Kelvin, which are converted to Celsius by multiplying the values by a scaling factor of 0.02 and then subtracting 273.15.

In [ ]:
import ee
import geemap
# Authenticate with Earth Engine
ee.Authenticate()
ee.Initialize(project='serene-bastion-406504')

In [ ]:
import ee
import pandas as pd
from datetime import datetime, timedelta

# Initialize the Earth Engine API
ee.Initialize(project='serene-bastion-406504')

# Define the region of interest (ROI)
roi = ee.FeatureCollection("projects/serene-bastion-406504/assets/Demo_Merged_Plots")

# Define a date range for March 2023
start_date = '2022-03-01'
end_date = '2022-07-31'

# Function to check the presence of MODIS LST images for a specific date
def check_modis_lst_presence(date):
    modis = ee.ImageCollection('MODIS/061/MOD11A1')
    mod11a1 = modis.filterDate(date, ee.Date(date).advance(1, 'day'))
    count = mod11a1.size().getInfo()
    return count > 0

# Function to sample MODIS LST at centroid for a specific date
def sample_modis_lst_at_centroid(feature, date):
    # Calculate the centroid of the feature
    centroid = feature.geometry().centroid()

    # Create a FeatureCollection with the centroid
    centroid_fc = ee.FeatureCollection([ee.Feature(centroid)])

    # Sample MODIS LST at the centroid for the specific date
    modis = ee.ImageCollection('MODIS/061/MOD11A1')
    mod11a1 = modis.filterDate(date, ee.Date(date).advance(1, 'day')).filterBounds(centroid)
    modLSTday = mod11a1.select('LST_Day_1km')

    def convert_to_celsius(img):
        celsius = img.multiply(0.02).subtract(273.15).copyProperties(img, ['system:time_start'])
        return celsius

    modLSTc = modLSTday.map(convert_to_celsius)

    # Sample MODIS LST at the centroid
    samples = modLSTc.getRegion(centroid_fc, scale=1000)

    return samples

# Initialize list to hold all sampled data
all_data = []

# Loop through each feature (plot) in the ROI
for feature in roi.toList(roi.size()).getInfo():
    feature = ee.Feature(feature)
    plot_name = feature.get('Plot_No').getInfo()  # Correctly extract plot name
    print(f"Processing plot {plot_name}...")

    # Loop through each date in the specified range
    date = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')

    while date <= end:
        date_str = date.strftime('%Y-%m-%d')

        # Check if MODIS LST images are available for the specific date
        if check_modis_lst_presence(date_str):
            # Sample MODIS LST at the centroid for the specific date
            try:
                samples = sample_modis_lst_at_centroid(feature, date_str)

                # Convert ee.List to Python list
                samples = samples.getInfo()

                # Check if samples were retrieved
                if samples:
                    # Convert the samples to a DataFrame
                    header = samples[0]
                    data = pd.DataFrame(samples[1:], columns=header)

                    # Extract relevant columns (longitude, latitude, LST, time)
                    df = data[['longitude', 'latitude', 'LST_Day_1km', 'time']]

                    # Rename columns
                    df.columns = ['Longitude', 'Latitude', 'LST_Celsius', 'Unix_Time']

                    # Convert Unix time to human-readable date
                    df['Date'] = pd.to_datetime(df['Unix_Time'], unit='ms').dt.strftime('%Y-%m-%d')

                    # Drop the Unix_Time column
                    df = df.drop(columns=['Unix_Time'])

                    # Drop rows with null LST_Celsius values
                    df = df.dropna(subset=['LST_Celsius'])

                    # Add plot number to the DataFrame
                    df['Plot_No'] = plot_name

                    # Append to all_data
                    all_data.append(df)
                else:
                    # If no samples, create an empty entry for that date
                    df = pd.DataFrame({
                        'Longitude': [None],
                        'Latitude': [None],
                        'LST_Celsius': [None],
                        'Date': [date_str],
                        'Plot_No': [plot_name]
                    })
                    all_data.append(df)
            except ee.EEException as e:
                print(f"No MODIS LST data found for plot {plot_name} on {date_str}: {e}")
                # Create an empty entry for that date
                df = pd.DataFrame({
                    'Longitude': [None],
                    'Latitude': [None],
                    'LST_Celsius': [None],
                    'Date': [date_str],
                    'Plot_No': [plot_name]
                })
                all_data.append(df)
        else:
            print(f"No MODIS LST images available for {date_str}.")
            # Create an empty entry for that date
            df = pd.DataFrame({
                'Longitude': [None],
                'Latitude': [None],
                'LST_Celsius': [None],
                'Date': [date_str],
                'Plot_No': [plot_name]
            })
            all_data.append(df)

        # Move to the next date
        date += timedelta(days=1)

# Concatenate all DataFrames into one
if all_data:
    df_all = pd.concat(all_data, ignore_index=True)

    # Calculate max and min LST values for each plot
    df_all['LST_Celsius'] = pd.to_numeric(df_all['LST_Celsius'], errors='coerce')
    max_lst = df_all.groupby('Plot_No')['LST_Celsius'].transform('max')
    min_lst = df_all.groupby('Plot_No')['LST_Celsius'].transform('min')

    # Add max and min LST values to the DataFrame
    df_all['Max_LST_Celsius'] = max_lst
    df_all['Min_LST_Celsius'] = min_lst

    # Save to CSV
    df_all.to_csv('Combined_MODIS_LST.csv', index=False)

    print("CSV file created successfully.")
else:
    print("No data sampled.")


Processing plot Plot1...
Processing plot Plot2...
Processing plot Plot3...
Processing plot Plot4...
Processing plot Plot5...
Processing plot Plot6...
Processing plot Plot7...
Processing plot Plot8...
Processing plot Plot9...


Processing plot Plot10...
CSV file created successfully.


<ipython-input-3-19df12aab16d>:135: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(all_data, ignore_index=True)
